In [74]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [75]:
processed_data = pd.read_csv("Processed_Online_Retail.csv")
X = processed_data.drop(columns=["CustomerID", "InvoiceNo", "CustomerLifetimeValue", "InvoiceDate", "Description"])
y_regression = processed_data["CustomerLifetimeValue"]
cat_features = ["StockCode", "Country"]


In [76]:
x_train, x_test, y_train, y_test = train_test_split(X, y_regression, test_size=0.2, random_state=42)

In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Concatenate, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
inputDim = X.shape[1]
autoencoder = Sequential([
    Dense(64, activation="relu", input_shape=(inputDim,)),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
    Dense(32, activation="relu"),
    Dense(64, activation="relu"),
    Dense(inputDim, activation="linear")
])
autoencoder.compile(optimizer="adam", loss="mse")
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
autoencoder.fit(X, X, epochs=50, batch_size=32, validation_split=0.2, callbacks=[es])


Epoch 1/50


c:\Users\Asus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9819/9819 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 4230.7080 - val_loss: 3143.9890
Epoch 2/50
9819/9819 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 300.6635 - val_loss: 7950.2534
Epoch 3/50
9819/9819 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 603.9515 - val_loss: 22262.3145
Epoch 4/50
9819/9819 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 402.6090 - val_loss: 15078.4854
Epoch 5/50
9819/9819 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 593.6121 - val_loss: 2187.8831
Epoch 6/50
9819/9819 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - loss: 3417.5312 - val_loss: 479.4383
Epoch 7/50
9819/9819 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 586.1953 - val_loss: 10201.5898
Epoch 8/50
9819/9819 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 895.0048 - val_loss: 1508.0980
Epoch 9/50
9819/9819 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - loss: 1039.2576 - val_loss: 2812.7642
Epoch 10/50
9819/9819 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 502.4495 - val_loss: 687.6629
Epoch 11/50
9819/9819 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 49

In [78]:
encoded_X = autoencoder.predict(X)
encoded_X = pd.DataFrame(encoded_X, columns=[f"enc_feature_{i}" for i in range(encoded_X.shape[1])])
compinedData = pd.concat([X, encoded_X], axis=1)
compinedData.head()


12273/12273 ━━━━━━━━━━━━━━━━━━━━ 7s 541us/step


,StockCode,Quantity,UnitPrice,Country,TotalPrice,Recency,Frequency,InvoiceHour,InvoiceDay,InvoiceMonth,...,enc_feature_1,enc_feature_2,enc_feature_3,enc_feature_4,enc_feature_5,enc_feature_6,enc_feature_7,enc_feature_8,enc_feature_9,enc_feature_10
0,3233,6,2.55,35,15.30,373,34,8,2,12,...,7.944555,5.982572,36.100388,25.778307,372.104279,32.321091,15.300911,3.207188,6.420877,0.261149
1,2643,6,3.39,35,20.34,373,34,8,2,12,...,7.223739,6.006860,34.593933,28.068228,372.644928,32.425224,14.405629,3.019927,5.619002,0.789011
2,2847,8,2.75,35,22.00,373,34,8,2,12,...,9.408938,5.750907,35.022408,30.419409,372.386353,32.467667,14.643950,2.953725,5.817553,0.619171
3,2795,6,3.39,35,20.34,373,34,8,2,12,...,7.284576,6.176656,34.890839,28.640947,372.547882,32.333183,14.563661,3.051500,5.755949,0.718737
4,2794,6,3.39,35,20.34,373,34,8,2,12,...,7.284328,6.175597,34.888767,28.637402,372.548737,32.333778,14.562592,3.051330,5.755115,0.719274


In [79]:
n_country = X["Country"].nunique()
n_stockcode = X["StockCode"].nunique()

In [80]:
from tensorflow.keras.models import Model
compinedData = compinedData.copy()
compinedData['Country_idx'], country_uniques = pd.factorize(compinedData['Country'])
compinedData['StockCode_idx'], stock_uniques = pd.factorize(compinedData['StockCode'])
n_country = len(country_uniques)
n_stockcode = len(stock_uniques)
country_emb_dim = 5
stock_emb_dim = 16
country_in = Input(shape=(1,), name='country_in')
stock_in = Input(shape=(1,), name='stock_in')
country_emb = Embedding(input_dim=n_country, output_dim=country_emb_dim, name='country_emb')(country_in)
country_emb = Flatten()(country_emb)
stock_emb = Embedding(input_dim=n_stockcode, output_dim=stock_emb_dim, name='stock_emb')(stock_in)
stock_emb = Flatten()(stock_emb)
emb_model = Model(inputs=[country_in, stock_in], outputs=Concatenate()([country_emb, stock_emb]))
emb_matrix = emb_model.predict([compinedData['Country_idx'].values, compinedData['StockCode_idx'].values])
country_emb_df = pd.DataFrame(emb_matrix[:, :country_emb_dim],columns=[f'country_emb_{i}' for i in range(5)], index=compinedData.index)
stock_emb_df = pd.DataFrame(emb_matrix[:, country_emb_dim:], columns=[f'stock_emb_{i}' for i in range(16)], index=compinedData.index)
compinedData = pd.concat([compinedData, country_emb_df, stock_emb_df], axis=1)
compinedData.drop(columns=['Country_idx', 'StockCode_idx'], inplace=True)
compinedData

12273/12273 ━━━━━━━━━━━━━━━━━━━━ 7s 603us/step


,StockCode,Quantity,UnitPrice,Country,TotalPrice,Recency,Frequency,InvoiceHour,InvoiceDay,InvoiceMonth,...,stock_emb_6,stock_emb_7,stock_emb_8,stock_emb_9,stock_emb_10,stock_emb_11,stock_emb_12,stock_emb_13,stock_emb_14,stock_emb_15
0,3233,6,2.55,35,15.30,373,34,8,2,12,...,-0.027349,-0.002559,0.031257,0.046948,0.038345,-0.022153,-0.005820,0.030831,-0.025611,0.044521
1,2643,6,3.39,35,20.34,373,34,8,2,12,...,-0.044017,0.001677,0.020899,-0.041760,0.042007,-0.046411,-0.002635,-0.039544,0.038528,-0.006487
2,2847,8,2.75,35,22.00,373,34,8,2,12,...,0.035080,0.029920,0.036721,-0.031448,0.005555,-0.031715,-0.027599,-0.042941,-0.022699,-0.015952
3,2795,6,3.39,35,20.34,373,34,8,2,12,...,-0.032402,-0.018723,-0.020236,-0.019042,0.037852,0.049584,0.006496,-0.011116,0.001327,0.044389
4,2794,6,3.39,35,20.34,373,34,8,2,12,...,-0.046609,-0.044637,-0.010765,0.024191,0.022789,-0.004748,0.004052,-0.009705,0.006255,-0.014653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392727,1466,12,0.85,13,10.20,0,4,12,4,12,...,-0.047139,0.041807,0.029218,-0.044652,-0.049623,-0.047122,-0.011024,0.032635,0.044276,-0.014023
392728,1742,6,2.10,13,12.60,0,4,12,4,12,...,-0.016225,0.017657,0.031093,0.023329,-0.025626,-0.018126,-0.033225,-0.024468,-0.043490,0.033793
392729,2082,4,4.15,13,16.60,0,4,12,4,12,...,-0.012995,-0.048142,-0.027005,-0.023808,0.007033,0.025539,0.014392,-0.021992,-0.049366,0.031373
392730,2083,4,4.15,13,16.60,0,4,12,4,12,...,-0.020712,-0.022152,-0.039316,-0.048133,-0.004085,-0.040732,-0.008045,-0.043398,0.044843,-0.033014


In [81]:
x_train, x_test, y_train, y_test = train_test_split(compinedData, y_regression, test_size=0.2, random_state=42)

In [85]:
from tensorflow.keras.layers import BatchNormalization, Dropout
DNNmodel = Sequential([
    Dense(128, activation='relu', input_shape=(compinedData.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='linear')
])
DNNmodel.compile(optimizer="Adam", loss='mae', metrics=['mae'])
es = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
DNNmodel.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[es])

Epoch 1/50


c:\Users\Asus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7855/7855 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 11102.0713 - mae: 11102.0713 - val_loss: 10173.0107 - val_mae: 10173.0107
Epoch 2/50
7855/7855 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 9482.3311 - mae: 9482.3311 - val_loss: 8862.7480 - val_mae: 8862.7480
Epoch 3/50
7855/7855 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 8535.5254 - mae: 8535.5254 - val_loss: 8130.6602 - val_mae: 8130.6602
Epoch 4/50
7855/7855 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 7946.1372 - mae: 7946.1372 - val_loss: 7682.1855 - val_mae: 7682.1855
Epoch 5/50
7855/7855 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 7625.0117 - mae: 7625.0117 - val_loss: 9330.8496 - val_mae: 9330.8496
Epoch 6/50
7855/7855 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 7484.2026 - mae: 7484.2026 - val_loss: 7595.1050 - val_mae: 7595.1050
Epoch 7/50
7855/7855 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 7465.8164 - mae: 7465.8164 - val_loss: 7169.8013 - val_mae: 7169.8013
Epoch 8/50
7855/7855 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 7388.3667 - m